In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ ||
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_.
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we
continue to use the CPU.



In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_.
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


--------------




Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3169, -0.1827,  0.6891,  0.1260,  0.1206, -0.1807, -0.0056,  0.1159,
          0.2746, -0.2001,  0.1075, -0.2499, -0.0840,  0.1612, -0.4389, -0.5185,
          0.2008,  0.3350,  0.0428, -0.0838],
        [-0.3469, -0.5164,  0.6360,  0.0764,  0.4927, -0.2406, -0.4095,  0.1764,
          0.5371,  0.1385,  0.3797,  0.3573, -0.2032, -0.0735, -0.3378, -0.0479,
          0.3043,  0.0942,  0.2784, -0.0807],
        [-0.0868, -0.5703,  0.6054,  0.3114,  0.1714, -0.2158,  0.1462, -0.2420,
          0.2305,  0.1009,  0.0495, -0.1303,  0.3630,  0.0029, -0.2067, -0.2610,
          0.1866,  0.1442,  0.5022,  0.1442]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.6891, 0.1260, 0.1206, 0.0000, 0.0000, 0.1159, 0.2746,
         0.0000, 0.1075, 0.0000, 0.0000, 0.1612, 0.0000, 0.0000, 0.2008, 0.3350,
         0.0428, 0.0000],
        [0.0000, 0.0000, 0.6360, 0.0764, 0.4927, 0.0000, 0.0000, 0.1764, 0.5371,
         0.1385, 0.3797, 0.3573, 0.0000, 0.0000, 0.00

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
pred_probab

tensor([[0.0570, 0.1222, 0.0970, 0.1001, 0.1066, 0.0807, 0.0852, 0.1100, 0.1344,
         0.1069],
        [0.0667, 0.1318, 0.1030, 0.1011, 0.1109, 0.0944, 0.0803, 0.0888, 0.1275,
         0.0955],
        [0.0687, 0.1272, 0.0944, 0.1002, 0.1062, 0.0976, 0.0835, 0.0915, 0.1256,
         0.1053]], grad_fn=<SoftmaxBackward0>)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0208,  0.0222,  0.0201,  ...,  0.0225,  0.0152,  0.0087],
        [-0.0191, -0.0070, -0.0096,  ...,  0.0321,  0.0107, -0.0342]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0295, 0.0063], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0022, -0.0401, -0.0395,  ..., -0.0398, -0.0009, -0.0160],
        [ 0.0378,  0.0405,  0.0085,  ...,  0.0356,  0.0106,  0.0264]],
       device='cuda:0', grad_fn=<Slic

--------------




Further Reading
--------------
- `torch.nn API <https://pytorch.org/docs/stable/nn.html>`_

